# Configuration interaction

Multipsi provides a very general configuration code, allowing a wide variety of CI expansions, including FullCI and truncated CI (for example CI with single and double excitations CISD). Note however that the code is mostly designed around Full CI (for use in particular in complete active space CAS methods, see following chapters) and is thus not as efficient as a dedicated CIS or CISD code.

## Full CI

Let's start with a simple FullCI of water.

MultiPsi relies on VeloxChem to define the molecule and basis set. For more informations about how to run Veloxchem, please go to the Veloxchem webpage.

> <https://veloxchem.org>

Here we will start from a simple Hartree-Fock calculation in Veloxchem:

In [1]:
import veloxchem as vlx

water_str='''
O   0.0   0.0   0.0
H   0.0   1.4   1.1
H   0.0  -1.4   1.1
'''
water=vlx.Molecule.read_str(water_str)
basis = vlx.MolecularBasis.read(water,"6-31G")
scfdrv = vlx.ScfRestrictedDriver()
hf_results = scfdrv.compute(water, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

We then define an orbital space object to store the starting orbitals and define the type of CI expansion:

In [2]:
import multipsi as mtp

space=mtp.OrbSpace(water,scfdrv.mol_orbs)
print(space)


               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 5
               Number of active orbitals:              0
               Number of virtual orbitals:             8

               This is a restricted Hartree-Fock wavefunction


By default, OrbSpace defines a restricted Hartree-Fock wavefunction (or restricted open-shell if the molecule is not a singlet). To define the full CI, we simply use the command `fci()`.

In [3]:
space.fci()
print(space)


               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 0
               Number of active orbitals:              13
               Number of virtual orbitals:             0

               This is a CASSCF wavefunction: CAS(10,13)


As we can see, internally the code assumes a complete active space, but since all orbitals are included in the active space, it is indeed a full CI.

We can also choose to freeze the core orbitals, as the correlation for core electrons tend to be constant and thus cancel out in relative energies.

In [4]:
space.fci(n_frozen=1)
print(space)


               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 1
               Number of active orbitals:              12
               Number of virtual orbitals:             0

               This is a CASSCF wavefunction: CAS(8,12)


This reduces significantly the number of determinants and thus the calculation time. It is sometimes useful to know in advance the number of determinants to see if the calculation is even feasible. For this we can create a CIExpansion object:

In [5]:
print(mtp.CIExpansion(space))


               CI expansion:
               -------------
               Number of determinants:      245025




While expansions as large as hundreds of billions of determinants can in principle be run using a large enough supercomputer, any expansion beyond a billion will be very expensive.

We can now run the configuration interaction:

In [6]:
CIdrv=mtp.CIDriver()
ci_results = CIdrv.compute(water,basis,space)

                                                                                                                          
                          Configuration Interaction Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 1
               Number of active orbitals:              12
               Number of virtual orbitals:             0

               This is a CASSCF wavefunction: CAS(8,12)

               CI expansion:
               -------------
               Number of determinants:      245025


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────────────────────────

If we are interested in several states and not just the ground state, we simply provide the number of states to the compute function:

In [7]:
ci_results = CIdrv.compute(water,basis,space, n_states = 3) #We want 3 states now

                                                                                                                          
                          Configuration Interaction Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 1
               Number of active orbitals:              12
               Number of virtual orbitals:             0

               This is a CASSCF wavefunction: CAS(8,12)

               CI expansion:
               -------------
               Number of determinants:      245025


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────────────────────────

At the end of the calculation, the CI Driver prints a few informations, but the key results can also be accessed directly from the return dictionary.

In [8]:
print("Ground state energy:",ci_results["energies"][0]) #Get the energy of a specific state, here state number 0
print("Array of all state energies:",ci_results["energies"]) #Get a python list of energies
print()
print("Ground state natural orbitals occupation numbers:",ci_results["natural_occupations"][0])

Ground state energy: -75.90215524166568
Array of all state energies: [-75.90215524 -75.84903068 -75.84010871]

Ground state natural orbitals occupation numbers: [1.98787234e+00 1.98051697e+00 1.72827796e+00 1.69442049e+00
 3.04282587e-01 2.65893737e-01 3.50022813e-04 2.62155960e-04
 1.79768836e-02 5.28475859e-03 5.09315295e-03 9.76894891e-03]


## Truncated CI

To define a truncated CI, one simply needs to define the truncated expansion in the OrbSpace object.

CIS and CISD expansions have a specific command, and higher order truncations can be done by using `CI(n)` with n the excitation order.

In [9]:
space=mtp.OrbSpace(water,scfdrv.mol_orbs)
space.cisd(n_frozen=1) #Equivalent to space.ci(2,n_frozen=1)

ci_results = CIdrv.compute(water,basis,space)

                                                                                                                          
                          Configuration Interaction Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 1
               Number of active orbitals:              12
               Number of virtual orbitals:             0

               This is a GASSCF wavefunction

                        Cumulated   Min cumulated    Max cumulated
               Space     orbitals      occupation       occupation
                   1            4               6                8
                   2           12               8                8


               CI expansion:
               -------------
               Number of determinants:      1425


                